In [1]:
import numpy as np
import torch
import random as rd
import torch.nn as nn

from tqdm import tqdm
from model import LPGNN
from generate_data import generate_random_linear_program, solve_linear_program, generate_and_solve_batches
from scipy.optimize import linprog

In [2]:
### PARAMETERS ###
num_constraints = 2
num_variables = 5
batch_size = 10
learning_rate = 0.003
N = 100
out_func = 'obj'

device = device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = LPGNN(num_constraints, num_variables).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 200
pbar = tqdm(range(epochs))

  0%|                                                                                                   | 0/200 [00:00<?, ?it/s]

In [3]:
data_train = []
for i in range(N):
    c, A, b, constraints, l, u, solution, feasibility = generate_and_solve_batches(batch_size,
                                                                                   num_variables,
                                                                                   num_constraints,
                                                                                   out_func)
    data_train.append([c, A, b, constraints, l, u, solution, feasibility])

In [4]:
def train(model, c, A, b, constraint, l, u, sol, feas, out_func, optimizer):
    optimizer.zero_grad()
    out = model(c, A, b, constraint, l, u, out_func)

    loss = nn.MSELoss()
    if out_func == 'feas':
        loss = loss(out, feas)

    elif out_func == 'obj':
        #loss = loss(out, (c[:, None, :] @ sol[:, :, None])[:,0])
        loss = loss(out[:,0], torch.sum(c * sol, dim=1))

    else:
        loss = loss(out, sol)

    loss.backward()
    optimizer.step()
    return loss

In [5]:
# training model
print("\n\n")
print("Training model")
print("\n\n")

for epoch in pbar:
    for batch in data_train:
        c, A, b, constraints, l, u, sol, feas = batch
        c = c.to(device)
        A = A.to(device)
        b = b.to(device)
        constraints = constraints.to(device)
        l = l.to(device)
        u = u.to(device)
        sol = sol.to(device)
        feas = feas
        loss = train(model, c, A, b, constraints, l, u, sol, feas, out_func, optimizer)
        pbar.set_description(f"%.8f" % loss)




Training model





0.00105518: 100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [13:49<00:00,  4.15s/it]


In [6]:
# Test model
print("\n\n")
print("Testing model")
print("\n\n")
batch_size = 5
M = 10

data_test = []
for i in range(M):
    c, A, b, constraints, l, u, solution, feasibility = generate_and_solve_batches(batch_size,
                                                                                   num_variables,
                                                                                   num_constraints,
                                                                                   out_func)
    data_test.append([c, A, b, constraints, l, u, solution, feasibility])

def test(model, c, A, b, constraint, l, u, sol, feas, out_func):
    out = model.forward(c, A, b, constraint, l, u, out_func)
    loss = nn.MSELoss()
    if out_func == 'feas':
        loss = loss(out, feas)
    elif out_func == 'obj':
        loss = loss(out, c.T @ sol)
    else:
        loss = loss(out, sol)
    return loss


for batch in data_test:
    c, A, b, constraints, l, u, sol, feas = batch
    c = c.to(device)
    A = A.to(device)
    b = b.to(device)
    constraints = constraints.to(device)
    l = l.to(device)
    u = u.to(device)
    sol = sol.to(device)
    feas = feas.to(device)

    loss = test(model, c, A, b, constraints, l, u, sol, feas, out_func)
    print(f"Loss: {loss}")

/home/wellington/.local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5, 5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)





Testing model



Loss: 0.03629964217543602
Loss: 0.026638749986886978
Loss: 0.021153492853045464
Loss: 0.01904143951833248
Loss: 0.027411531656980515
Loss: 0.02739361859858036
Loss: 0.037968821823596954
Loss: 0.021481657400727272
Loss: 0.0209252517670393
Loss: 0.015586357563734055


In [7]:
model2 = LPGNN(num_constraints, num_variables).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

# Test model
print("\n\n")
print("Testing model")
print("\n\n")
batch_size = 5
M = 10

data_test = []
for i in range(M):
    c, A, b, constraints, l, u, solution, feasibility = generate_and_solve_batches(batch_size,
                                                                                   num_variables,
                                                                                   num_constraints,
                                                                                   out_func)
    data_test.append([c, A, b, constraints, l, u, solution, feasibility])

def test(model, c, A, b, constraint, l, u, sol, feas, out_func):
    out = model.forward(c, A, b, constraint, l, u, out_func)
    loss = nn.MSELoss()
    if out_func == 'feas':
        loss = loss(out, feas)
    elif out_func == 'obj':
        loss = loss(out, c.T @ sol)
    else:
        loss = loss(out, sol)
    return loss


for batch in data_test:
    c, A, b, constraints, l, u, sol, feas = batch
    c = c.to(device)
    A = A.to(device)
    b = b.to(device)
    constraints = constraints.to(device)
    l = l.to(device)
    u = u.to(device)
    sol = sol.to(device)
    feas = feas.to(device)

    loss = test(model2, c, A, b, constraints, l, u, sol, feas, out_func)
    print(f"Loss: {loss}")




Testing model



Loss: 0.3098640441894531
Loss: 0.4725342392921448
Loss: 0.45849043130874634
Loss: 0.45237258076667786
Loss: 0.5035025477409363
Loss: 0.43440669775009155
Loss: 0.506040096282959
Loss: 0.34443971514701843
Loss: 0.3126998543739319
Loss: 0.3500613272190094
